### Loading and Saving State of Agents

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

import os

from dotenv import load_dotenv

load_dotenv()

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("deployment_name"),
    model=os.getenv("model_name"),
    api_version=os.getenv("api_version"),
    azure_endpoint=os.getenv("azure_endpoint"),
    api_key=os.getenv("api_key")
)

In [ ]:
assistant_agent = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are a fun fact expert about animals.",
)

response = await assistant_agent.on_messages(
    [
        TextMessage(
            content="Tell me a fun fact about dolphins.",
            source="user"
        )
    ],
    CancellationToken()
)

In [ ]:
print(response.chat_message.content)

In [ ]:
agent_state = await assistant_agent.save_state()

print("Agent state saved successfully.")
print(agent_state)

In [ ]:
new_assistant_agent = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are a fun fact expert about animals.",
)

await new_assistant_agent.load_state(agent_state)

response = await new_assistant_agent.on_messages(
    [
        TextMessage(
            content="what was the fun fact you shared with me about dolphins?",
            source="user"
        )
    ],
    CancellationToken()
)

In [ ]:
print(response.chat_message.content)

In [ ]:
assistant_agent = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are a helpful assistant, who has abilities to fun facts about animals.",
)

agent_team = RoundRobinGroupChat(
    [assistant_agent],
    termination_condition=MaxMessageTermination(5),
)

stream = agent_team.run_stream(
    task="Tell me a fun fact about Octopuses.",
)

await Console(stream)

In [ ]:
team_state = await agent_team.save_state()
print("Team state saved successfully.")
print(team_state)

In [ ]:
await agent_team.reset()

In [ ]:
stream = agent_team.run_stream(
    task="what was the last animal fact you have shared with me?",
)

await Console(stream)

In [ ]:
await agent_team.load_state(team_state)

stream = agent_team.run_stream(
    task="what was the last animal fact you have shared with me?",
)

await Console(stream)

In [ ]:
import json

with open("team_state.json", "w") as f:
    json.dump(team_state, f, indent=4, sort_keys=True, default=str)

In [ ]:
new_agent_team = RoundRobinGroupChat(
    [assistant_agent],
    termination_condition=MaxMessageTermination(5),
)

with open("team_state.json", "r") as f:
    team_state = json.load(f)

await new_agent_team.load_state(team_state)

stream = new_agent_team.run_stream(
    task="what was the last animal fact you have shared with me?",
)

await Console(stream)